In [27]:
#pip install opencv-contrib-python --user

import cv2
import os
import numpy as np

def face_recog_train():
    dataset_path = 'dataset/'
    dirs = os.listdir(dataset_path)#dataset 하위 디렉토리 이름을 리스트에 저장
    persons = []
    for dir in dirs: #각 디렉토리에 저장된 파일명들을 persons에 담음
        if os.path.isdir(dataset_path+dir):
            files = os.listdir(dataset_path+dir)
            for idx, f in enumerate(files):
                files[idx] = dataset_path+dir+'/'+f
            persons.append(files)
    print(persons)
    
    classifiers = cv2.CascadeClassifier('haar/haarcascade_frontalface_alt2.xml')#얼굴 분류기
    recognizer = cv2.face.LBPHFaceRecognizer_create() # 얼굴 인식기 객체
    samples = []  # 학습 데이터
    ids = []  # 레이블(정답)
    #학습할 얼굴 샘플링
    for id, row in enumerate(persons):  #id:0, row:[정우성 사진들] / id:1, row:[이나영 사진들]
        for p in row:
            img = cv2.imread(p)
            print(p)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            face = classifiers.detectMultiScale(
                gray,
                scaleFactor=1.2,
                minNeighbors=5,
                minSize=(20, 20)
            )
            for (x,y,w,h) in face:
                samples.append(gray[y:y+h, x:x+w])
                ids.append(id)

    recognizer.train(samples, np.array(ids))
    recognizer.write('trainer.yml')
    print('얼굴 학습 완료')

In [28]:
face_recog_train()

[['dataset/p1/AnyConv.com__1.jpg', 'dataset/p1/AnyConv.com__10.jpg', 'dataset/p1/AnyConv.com__2.jpg', 'dataset/p1/AnyConv.com__3.jpg', 'dataset/p1/AnyConv.com__4.jpg', 'dataset/p1/AnyConv.com__6.jpg', 'dataset/p1/AnyConv.com__7.jpg', 'dataset/p1/AnyConv.com__8.jpg', 'dataset/p1/AnyConv.com__9.jpg'], ['dataset/p2/AnyConv.com__1.jpg', 'dataset/p2/AnyConv.com__10.jpg', 'dataset/p2/AnyConv.com__2.jpg', 'dataset/p2/AnyConv.com__3.jpg', 'dataset/p2/AnyConv.com__4.jpg', 'dataset/p2/AnyConv.com__5.jpg', 'dataset/p2/AnyConv.com__6.jpg', 'dataset/p2/AnyConv.com__7.jpg', 'dataset/p2/AnyConv.com__8.jpg', 'dataset/p2/AnyConv.com__9.jpg'], ['dataset/p3/AnyConv.com_ (1).jpeg', 'dataset/p3/AnyConv.com_ (1).jpg', 'dataset/p3/AnyConv.com_ (2).jpg', 'dataset/p3/AnyConv.com_ (3).jpg', 'dataset/p3/AnyConv.com_ (4).jpg', 'dataset/p3/AnyConv.com_ (5).jpg', 'dataset/p3/AnyConv.com_ (6).jpg', 'dataset/p3/AnyConv.com_ (7).jpg', 'dataset/p3/AnyConv.com_ (8).jpg']]
dataset/p1/AnyConv.com__1.jpg
dataset/p1/AnyConv

In [35]:
def face_recog():

    classifiers = cv2.CascadeClassifier('haar/haarcascade_frontalface_alt2.xml')
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    
    recognizer.read('trainer.yml')

    names=['정우성', '이나영', '고창석']
    img = cv2.imread('dataset/test1.jpg')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face = classifiers.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=5,
        minSize=(20, 20)
    )
    res = False
    name = 'no face'
    for (x, y, w, h) in face:
        print('얼굴감지')
        id, confi = recognizer.predict(gray[y:y+h, x:x+w])
        #if confi < 50:
        name = names[id]
        print(name,'/ confidence:', confi)
        res = True
        
    if len(face)==0:
        name = 'unknown'
        res = False

    return res, name


In [36]:
face_recog()

(False, 'unknown')

In [1]:
import cv2

c = cv2.CascadeClassifier('haar/strawberrydetector.xml')
img = cv2.imread('img/s2.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
straw = c.detectMultiScale(gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(40, 40))

img2 = img.copy()
for (x, y, w, h) in straw:
    #print(x)
    cv2.rectangle(img2, (x, y), (x + w, y + h), (255, 0, 0), 2)

if len(straw)==0:
    print('no strawberry')
else:
    cv2.imshow('img', img)
    cv2.imshow('res', img2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [1]:
import cv2
import os
from pytesseract import *

path = 'img/car/'
files = os.listdir(path)
src = []
imgs = []

for f in files:
    img = cv2.imread(path+f)#이미지 로드
    img = cv2.resize(img, (400, 300))   
    src.append(img.copy())
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)#흑백처리
    blur = cv2.GaussianBlur(gray, (3,3), 0)#노이즈 제거
    _, dst = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)#쓰레숄딩
    imgs.append(dst)
    
for idx, th in enumerate(imgs):
    cont, _ = cv2.findContours(th, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)#외곽선 검출

    for c in cont:
        
        if cv2.contourArea(c) < 10000 or cv2.contourArea(c) > 30000:  #  너무 작으면 무시
            continue

        approx = cv2.approxPolyDP(c, cv2.arcLength(c, True)*0.02, True)
        vtc = len(approx)
        
        if vtc==4:
            print('탐지')
            x,y,w,h = cv2.boundingRect(c)
            cv2.rectangle(src[idx], (x, y, w, h), (0, 255, 255))
            
            roi = src[idx][y:y+h,x:x+w, :]
            cv2.imshow('img_'+str(idx), roi)
            #text = pytesseract.image_to_string(roi,lang='kor')
            #print(text+" 차량 입차")
        
    #cv2.imshow('img'+str(idx), src[idx])
cv2.waitKey()
cv2.destroyAllWindows()

탐지
탐지


텍슽

In [7]:
from PIL import Image
from pytesseract import *

filename = "img/sampletxt2.jpg"
image = Image.open(filename)
text = image_to_string(image, lang='kor')
print(text)

원 전 지사는 이날
후 6시까지 작성한 녹취록이 아닌 녹음파일 전체를 공개하라"고 압박
날 발 늦게 일부 녹취록을

구한 것이다.

날 오전 9시 여의도 당사에서 긴급 기자회견을 열고 "이 대표는 오늘
헝     | 다
록을 공개하며 반격에 나서자 녹음파일 전체를 모두 공개하라고 요

지
록 일부를 공개했다"면서 "제 기역과 양

발언의 대상은 윤석열 후보"라고 주장했다.

|사는 "이 대표는 파문이 확산하고
침

